In [1]:
from zipline.pipeline.fundamentals import Fundamentals
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline

## 辅助数据

财务报告公告日期

In [2]:
from cswd.tasks.report_dates import BookReportDate

In [3]:
brd = BookReportDate()
dates = brd.data

In [4]:
dates.shape

(97799, 9)

## 预处理的公告日期

采集的财务报告公告日期存放在本地文件

In [5]:
from zipline.pipeline.fundamentals.financal import get_preprocessed_dates

In [6]:
import pandas as pd

In [7]:
df = get_preprocessed_dates()
df = df[df.report_end_date >= pd.Timestamp('2016-12-31').date()]

In [8]:
df[df.code.isin(['000333','600645'])].sort_values(['code','report_end_date'])

,report_end_date,timestamp,code
1296,2016-12-31,2017-03-31,000333
1327,2017-03-31,2017-04-29,000333
1388,2017-06-30,2017-08-31,000333
1433,2017-09-30,2017-10-31,000333
1483,2017-12-31,2018-03-31,000333
1490,2018-03-31,2018-04-28,000333
532,2016-12-31,2017-04-29,600645
532,2017-03-31,2017-04-29,600645
532,2017-06-30,2017-08-31,600645
532,2017-09-30,2017-10-31,600645


## `pipeline`

In [9]:
def make_pipeline():

    eps = Fundamentals.profit_statement.A044.latest

    return Pipeline(columns={
        '基本每股收益': eps,
        '报告日期': Fundamentals.profit_statement.report_end_date.latest,
        '公告日期':Fundamentals.profit_statement.asof_date.latest,
    })

In [10]:
result = run_pipeline(make_pipeline(), '2018-04-24', '2018-05-02')

In [11]:
select_output_by(result, stock_codes=['600645','000333'])

公告日期  基本每股收益       报告日期
2018-04-24 00:00:00+00:00 美的集团(000333) 2018-03-30    2.66 2017-12-31
                          中源协和(600645) 2017-10-30    0.01 2017-09-30
2018-04-25 00:00:00+00:00 美的集团(000333) 2018-03-30    2.66 2017-12-31
                          中源协和(600645) 2017-10-30    0.01 2017-09-30
2018-04-26 00:00:00+00:00 美的集团(000333) 2018-03-30    2.66 2017-12-31
                          中源协和(600645) 2017-10-30    0.01 2017-09-30
2018-04-27 00:00:00+00:00 美的集团(000333) 2018-03-30    2.66 2017-12-31
                          中源协和(600645) 2018-04-25    0.30 2018-03-31
2018-05-02 00:00:00+00:00 美的集团(000333) 2018-04-27    0.80 2018-03-31
                          中源协和(600645) 2018-04-25    0.30 2018-03-31

**由于年报与第一季度报告同时公布，`pipeline`处理时有断层。即会跳过年报，至今到三季度**